* 自建 vgg19 參考 https://gist.github.com/baraldilorenzo/8d096f48a1be4a2d660d
* vgg19 的預處理法 caffe https://medium.com/@sci218mike/%E5%9C%96%E7%89%87%E9%A0%90%E8%99%95%E7%90%86%E4%BD%BF%E7%94%A8keras-applications-%E7%9A%84-preprocess-input-6ef0963a483e

# model

In [2]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Input
from keras.layers import Conv2D, MaxPooling2D, InputLayer
from keras.optimizers import SGD

def generateVGG19(input_tensor):
    model = Sequential()
    
    model.add(InputLayer(input_tensor = input_tensor))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))
    
    weights_path = 'vgg19_weights_notop.h5'
    model.load_weights(weights_path, by_name=True)

    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')

    return model


# functions

In [1]:
from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
from scipy.misc import imsave
from keras.applications import vgg19
from keras import backend as K
import os
from PIL import Image, ImageFont, ImageDraw, ImageOps, ImageEnhance, ImageFilter

base_image_path = 'targets/rooftop.jpg'
style_reference_image_path = 'styles/starry night.jpg'
results_path = 'results/'
iterations = 200
pictrue_size = 440

source_image = Image.open(base_image_path)
source_image= source_image.resize((pictrue_size, pictrue_size))
width, height = pictrue_size, pictrue_size

def save_img(fname, image, image_enhance=True):  # 圖像增強
    image = Image.fromarray(image)
    if image_enhance:
        # 亮度增強
        enh_bri = ImageEnhance.Brightness(image)
        brightness = 1.2
        image = enh_bri.enhance(brightness)
        # 色度增強
        enh_col = ImageEnhance.Color(image)
        color = 1.2
        image = enh_col.enhance(color)
        # 銳度增強
        enh_sha = ImageEnhance.Sharpness(image)
        sharpness = 1.2
        image = enh_sha.enhance(sharpness)
    imsave(fname, image)
    return

def preprocess_image(image):
    """
    預處理圖片，包括變形到(1，width, height)形狀，數據歸一到0-1之間
    :param image: 輸入一張圖片
    :return: 預處理好的圖片
    """
    image = image.resize((width, height))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)  # (width, height)->(1，width, height)
    image = vgg19.preprocess_input(image)  # 0-255 -> 0-1.0
    return image

def deprocess_image(x):
    """
    將0-1之間的數據變成圖片的形式返回
    :param x: 數據在0-1之間的矩陣
    :return: 圖片，數據都在0-255之間
    """
    x = x.reshape((width, height, 3))
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')  # 以防超出255範圍
    return x

# Gram矩陣
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 風格損失，是風格圖片與結果圖片的 Gram矩陣之差，並對所有元素求和
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    S_C = S-C
    channels = 3
    size = height * width
    return K.sum(K.square(S_C)) / (4. * (channels ** 2) * (size ** 2))
    #return K.sum(K.pow(S_C,4)) / (4. * (channels ** 2) * (size ** 2))  # 居然和平方沒有什麼不同
    #return K.sum(K.pow(S_C,4)+K.pow(S_C,2)) / (4. * (channels ** 2) * (size ** 2))  # 也能用，花後面出現了葉子

# 輸入x，輸出對應於 x的梯度和loss
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, height, width))
    else:
        x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])  # 輸入x，得到輸出
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent
def total_variation_loss(x,img_nrows=width, img_ncols=height):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


# Evaluator可以只需要進行一次計算就能得到所有的梯度和loss
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

Using TensorFlow backend.


In [3]:
# 得到需要處理的數據，處理爲keras的變量（tensor），處理爲一個(3, width, height, 3)的矩陣
# 分別是基準圖片，風格圖片，結果圖片
base_image = K.variable(preprocess_image(source_image))   # 基準圖像
style_reference_image = K.variable(preprocess_image(load_img(style_reference_image_path)))
combination_image = K.placeholder((1, width, height, 3))
# 組合以上3張圖片，作爲一個 keras輸入向量
input_tensor = K.concatenate([base_image, style_reference_image, combination_image], axis=0)


model = generateVGG19(input_tensor=input_tensor)
# model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet',include_top=False)
print("Model loaded.")

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

loss = K.variable(0.)

layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
content_weight = 0.08
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

feature_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
feature_layers_w = [0.1,0.1,0.4,0.3,0.1]

for i in range(len(feature_layers)):
    # 每一層的權重以及數據
    layer_name, w = feature_layers[i], feature_layers_w[i]
    layer_features = outputs_dict[layer_name]  # 該層的特徵

    style_reference_features = layer_features[1, :, :, :]  # 參考圖像在VGG網絡中第i層的特徵
    combination_features = layer_features[2, :, :, :]     # 結果圖像在VGG網絡中第i層的特徵

    loss += w * style_loss(style_reference_features, combination_features)  # 目標風格圖像的特徵和結果圖像特徵之間的差異作爲loss

loss += total_variation_loss(combination_image)


# 求得梯度，輸入combination_image，對loss求梯度, 每輪迭代中combination_image會根據梯度方向做調整
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

evaluator = Evaluator()
x = preprocess_image(source_image)
img = deprocess_image(x.copy())
fname = results_path + '原始圖片.png'
save_img(fname, img)

# 開始迭代
for i in range(iterations):
    start_time = time.time()
    print('iteration', i,end="   ")
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20, epsilon=1e-7)
    # 一個scipy的L-BFGS優化器
    print('目前loss:', min_val,end="  ")
    # 保存生成的圖片
    img = deprocess_image(x.copy())

    fname = 'result_%d.png' % i
    end_time = time.time()
    print('耗時%.2f s' % (end_time - start_time))

    if i%5 == 0 or i == iterations-1:
        save_img(results_path + fname, img, image_enhance=True)
        print('文件保存爲', fname)













Model loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\123na\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


iteration 0   目前loss: 4752128500.0  耗時242.78 s
文件保存爲 result_0.png
iteration 1   目前loss: 2323070000.0  耗時245.12 s
iteration 2   目前loss: 1735836200.0  耗時265.19 s
iteration 3   目前loss: 1517142700.0  耗時247.48 s
iteration 4   目前loss: 1381652000.0  耗時243.48 s
iteration 5   目前loss: 1303885400.0  耗時268.14 s
文件保存爲 result_5.png
iteration 6   

KeyboardInterrupt: 